In [ ]:
import math
import numpy as np
import secrets
import sys, os
from typing import Optional, Set, Tuple, Generator

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'python')))
from qfl_crypto import number_theory 

In [ ]:
def rwh_primes(n):
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Returns  a list of primes < n """
    sieve = [True] * n
    for i in range(3, int(n**0.5)+1,2):
        if sieve[i]:
            sieve[i*i::2*i]=[False]*((n-i*i-1)/(2*i)+1)
    return [2] + [i for i in range(3,n,2) if sieve[i]]

def rwh_primes1(n):
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Returns  a list of primes < n """
    sieve = [True] * (n/2)
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i/2]:
            sieve[i*i/2::i] = [False] * ((n-i*i-1)/(2*i)+1)
    return [2] + [2*i+1 for i in range(1,n/2) if sieve[i]]

def rwh_primes2(n):
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Input n>=6, Returns a list of primes, 2 <= p < n """
    correction = (n % 6 > 1)
    n = {0:n,1:n-1,2:n+4,3:n+3,4:n+2,5:n+1}[n % 6]
    sieve = [True] * (n/3)
    sieve[0] = False
    for i in range(n ** 0.5 // 3 + 1):
        if sieve[i]:
            k=3*i+1|1
            sieve[      ((k*k)/3)      ::2*k]=[False]*((n/6-(k*k)/6-1)/k+1)
            sieve[(k*k+4*k-2*k*(i&1))/3::2*k]=[False]*((n/6-(k*k+4*k-2*k*(i&1))/6-1)/k+1)
    return [2,3] + [3*i+1|1 for i in range(1,n/3-correction) if sieve[i]]

def sieve_wheel_30(N):
    # http://zerovolt.com/?p=88
    ''' Returns a list of primes <= N using wheel criterion 2*3*5 = 30

Copyright 2009 by zerovolt.com
This code is free for non-commercial purposes, in which case you can just leave this comment as a credit for my work.
If you need this code for commercial purposes, please contact me by sending an email to: info [at] zerovolt [dot] com.'''
    __smallp = ( 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
    61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139,
    149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227,
    229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311,
    313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401,
    409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491,
    499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599,
    601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683,
    691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797,
    809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887,
    907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997)

    wheel = (2, 3, 5)
    const = 30
    if N < 2:
        return []
    if N <= const:
        pos = 0
        while __smallp[pos] <= N:
            pos += 1
        return list(__smallp[:pos])
    # make the offsets list
    offsets = (7, 11, 13, 17, 19, 23, 29, 1)
    # prepare the list
    p = [2, 3, 5]
    dim = 2 + N // const
    tk1  = [True] * dim
    tk7  = [True] * dim
    tk11 = [True] * dim
    tk13 = [True] * dim
    tk17 = [True] * dim
    tk19 = [True] * dim
    tk23 = [True] * dim
    tk29 = [True] * dim
    tk1[0] = False
    # help dictionary d
    # d[a , b] = c  ==> if I want to find the smallest useful multiple of (30*pos)+a
    # on tkc, then I need the index given by the product of [(30*pos)+a][(30*pos)+b]
    # in general. If b < a, I need [(30*pos)+a][(30*(pos+1))+b]
    d = {}
    for x in offsets:
        for y in offsets:
            res = (x*y) % const
            if res in offsets:
                d[(x, res)] = y
    # another help dictionary: gives tkx calling tmptk[x]
    tmptk = {1:tk1, 7:tk7, 11:tk11, 13:tk13, 17:tk17, 19:tk19, 23:tk23, 29:tk29}
    pos, prime, lastadded, stop = 0, 0, 0, int(np.ceil(np.sqrt(N)))
    # inner functions definition
    def del_mult(tk, start, step):
        for k in range(start, len(tk), step):
            tk[k] = False
    # end of inner functions definition
    cpos = const * pos
    while prime < stop:
        # 30k + 7
        if tk7[pos]:
            prime = cpos + 7
            p.append(prime)
            lastadded = 7
            for off in offsets:
                tmp = d[(7, off)]
                start = (pos + prime) if off == 7 else (prime * (const * (pos + 1 if tmp < 7 else 0) + tmp) )//const
                del_mult(tmptk[off], start, prime)
        # 30k + 11
        if tk11[pos]:
            prime = cpos + 11
            p.append(prime)
            lastadded = 11
            for off in offsets:
                tmp = d[(11, off)]
                start = (pos + prime) if off == 11 else (prime * (const * (pos + 1 if tmp < 11 else 0) + tmp) )//const
                del_mult(tmptk[off], start, prime)
        # 30k + 13
        if tk13[pos]:
            prime = cpos + 13
            p.append(prime)
            lastadded = 13
            for off in offsets:
                tmp = d[(13, off)]
                start = (pos + prime) if off == 13 else (prime * (const * (pos + 1 if tmp < 13 else 0) + tmp) )//const
                del_mult(tmptk[off], start, prime)
        # 30k + 17
        if tk17[pos]:
            prime = cpos + 17
            p.append(prime)
            lastadded = 17
            for off in offsets:
                tmp = d[(17, off)]
                start = (pos + prime) if off == 17 else (prime * (const * (pos + 1 if tmp < 17 else 0) + tmp) )//const
                del_mult(tmptk[off], start, prime)
        # 30k + 19
        if tk19[pos]:
            prime = cpos + 19
            p.append(prime)
            lastadded = 19
            for off in offsets:
                tmp = d[(19, off)]
                start = (pos + prime) if off == 19 else (prime * (const * (pos + 1 if tmp < 19 else 0) + tmp) )//const
                del_mult(tmptk[off], start, prime)
        # 30k + 23
        if tk23[pos]:
            prime = cpos + 23
            p.append(prime)
            lastadded = 23
            for off in offsets:
                tmp = d[(23, off)]
                start = (pos + prime) if off == 23 else (prime * (const * (pos + 1 if tmp < 23 else 0) + tmp) )//const
                del_mult(tmptk[off], start, prime)
        # 30k + 29
        if tk29[pos]:
            prime = cpos + 29
            p.append(prime)
            lastadded = 29
            for off in offsets:
                tmp = d[(29, off)]
                start = (pos + prime) if off == 29 else (prime * (const * (pos + 1 if tmp < 29 else 0) + tmp) )//const
                del_mult(tmptk[off], start, prime)
        # now we go back to top tk1, so we need to increase pos by 1
        pos += 1
        cpos = const * pos
        # 30k + 1
        if tk1[pos]:
            prime = cpos + 1
            p.append(prime)
            lastadded = 1
            for off in offsets:
                tmp = d[(1, off)]
                start = (pos + prime) if off == 1 else (prime * (const * pos + tmp) )//const
                del_mult(tmptk[off], start, prime)
    # time to add remaining primes
    # if lastadded == 1, remove last element and start adding them from tk1
    # this way we don't need an "if" within the last while
    if lastadded == 1:
        p.pop()
    # now complete for every other possible prime
    while pos < len(tk1):
        cpos = const * pos
        if tk1[pos]: p.append(cpos + 1)
        if tk7[pos]: p.append(cpos + 7)
        if tk11[pos]: p.append(cpos + 11)
        if tk13[pos]: p.append(cpos + 13)
        if tk17[pos]: p.append(cpos + 17)
        if tk19[pos]: p.append(cpos + 19)
        if tk23[pos]: p.append(cpos + 23)
        if tk29[pos]: p.append(cpos + 29)
        pos += 1
    # remove exceeding if present
    pos = len(p) - 1
    while p[pos] > N:
        pos -= 1
    if pos < len(p) - 1:
        del p[pos+1:]
    # return p list
    return p

def sieveOfEratosthenes(n):
    """sieveOfEratosthenes(n): return the list of the primes < n."""
    # Code from: <dickinsm@gmail.com>, Nov 30 2006
    # http://groups.google.com/group/comp.lang.python/msg/f1f10ced88c68c2d
    if n <= 2:
        return []
    sieve = [i for i in range(3, n, 2)]
    len_sieve = len(sieve)
    for si in sieve:
        if si:
            next_si_multiple_index = (si*si - 3) // 2
            if next_si_multiple_index >= len_sieve:
                break
            number_of_si_multiples = -((next_si_multiple_index - len_sieve) // si)
            # From next_si_multiple_index, zero out every si entry
            sieve[next_si_multiple_index::si] = [0] * number_of_si_multiples
    return [2] + [el for el in sieve if el]

def sieveOfAtkin(end):
    """sieveOfAtkin(end): return a list of all the prime numbers <end
    using the Sieve of Atkin."""
    # Code by Steve Krenzel, <Sgk284@gmail.com>, improved
    # Code: https://web.archive.org/web/20080324064651/http://krenzel.info/?p=83
    # Info: http://en.wikipedia.org/wiki/Sieve_of_Atkin
    assert end > 0
    lng = ((end-1) // 2)
    sieve = [False] * (lng + 1)

    x_max, x2, xd = int(np.sqrt((end-1)/4.0)), 0, 4
    for xd in range(4, 8*x_max + 2, 8):
        x2 += xd
        y_max = int(np.sqrt(end-x2))
        n, n_diff = x2 + y_max*y_max, (y_max << 1) - 1
        if not (n & 1):
            n -= n_diff
            n_diff -= 2
        for d in range((n_diff - 1) << 1, -1, -8):
            m = n % 12
            if m == 1 or m == 5:
                m = n >> 1
                sieve[m] = not sieve[m]
            n -= d

    x_max, x2, xd = int(np.sqrt((end-1) / 3.0)), 0, 3
    for xd in range(3, 6 * x_max + 2, 6):
        x2 += xd
        y_max = int(np.sqrt(end-x2))
        n, n_diff = x2 + y_max*y_max, (y_max << 1) - 1
        if not(n & 1):
            n -= n_diff
            n_diff -= 2
        for d in range((n_diff - 1) << 1, -1, -8):
            if n % 12 == 7:
                m = n >> 1
                sieve[m] = not sieve[m]
            n -= d

    x_max, y_min, x2, xd = int((2 + np.sqrt(4-8*(1-end)))/4), -1, 0, 3
    for x in range(1, x_max + 1):
        x2 += xd
        xd += 6
        if x2 >= end: y_min = (((int(np.ceil(np.sqrt(x2 - end))) - 1) << 1) - 2) << 1
        n, n_diff = ((x*x + x) << 1) - 1, (((x-1) << 1) - 2) << 1
        for d in range(n_diff, y_min, -8):
            if n % 12 == 11:
                m = n >> 1
                sieve[m] = not sieve[m]
            n += d

    primes = [2, 3]
    if end <= 3:
        return primes[:max(0,end-2)]

    for n in range(5 >> 1, (int(np.sqrt(end))+1) >> 1):
        if sieve[n]:
            primes.append((n << 1) + 1)
            aux = (n << 1) + 1
            aux *= aux
            for k in range(aux, end, 2 * aux):
                sieve[k >> 1] = False

    s  = int(np.sqrt(end)) + 1
    if s  % 2 == 0:
        s += 1
    primes.extend([i for i in range(s, end, 2) if sieve[i >> 1]])

    return primes

def ambi_sieve_plain(n):
    s = range(3, n, 2)
    for m in range(3, int(n**0.5)+1, 2): 
        if s[(m-3)/2]: 
            for t in range((m*m-3)/2,(n>>1)-1,m):
                s[t]=0
    return [2]+[t for t in s if t>0]

def sundaram3(max_n):
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/2073279#2073279
    numbers = range(3, max_n+1, 2)
    half = (max_n)//2
    initial = 4

    for step in range(3, max_n+1, 2):
        for i in range(initial, half, step):
            numbers[i-1] = 0
        initial += 2*(step+1)

        if initial > half:
            return [2] + filter(None, numbers)

################################################################################
# Using Numpy:
def ambi_sieve(n):
    # http://tommih.blogspot.com/2009/04/fast-prime-number-generator.html
    s = np.arange(3, n, 2)
    for m in range(3, int(n ** 0.5)+1, 2): 
        if s[(m-3)/2]: 
            s[(m*m-3)/2::m]=0
    return np.r_[2, s[s>0]]

def primesfrom3to(n):
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Returns a array of primes, p < n """
    assert n>=2
    sieve = np.ones(n // 2, dtype=bool)
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i/2]:
            sieve[i*i/2::i] = False
    return np.r_[2, 2*np.nonzero(sieve)[0][1::]+1]    

def primesfrom2to(n):
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Input n>=6, Returns an array of primes, 2 <= p < n """
    assert n >= 6
    sieve = np.ones(n // 3 + (n % 6 == 2), dtype=bool)
    sieve[0] = False
    for i in range(int(n ** 0.5) // 3 + 1):
        if sieve[i]:
            k = 3 * i + 1 | 1
            sieve[((k * k) // 3)::2 * k] = False
            sieve[(k * k + 4 * k - 2 * k * (i & 1)) // 3::2 * k] = False
    return np.r_[2, 3, ((3 * np.nonzero(sieve)[0] + 1) | 1)]

def primesfrom2to_updated(n):
    # http://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    #""" Input N>=6, Returns a list of primes, 2 <= p < N """
    correction = n % 6 > 1
    N = {0:n, 1:n-1, 2:n+4, 3:n+3, 4:n+2, 5:n+1}[n%6]
    sieve = [True] * (N // 3)
    sieve[0] = False
    for i in range(int(N ** .5) // 3 + 1):
        if sieve[i]:
            k = (3 * i + 1) | 1
            sieve[k*k // 3::2*k] = [False] * ((N//6 - (k*k)//6 - 1)//k + 1)
            sieve[(k*k + 4*k - 2*k*(i%2)) // 3::2*k] = [False] * ((N // 6 - (k*k + 4*k - 2*k*(i%2))//6 - 1) // k + 1)
    return [2, 3] + [(3 * i + 1) | 1 for i in range(1, N // 3 - correction) if sieve[i]]


## 1.	Consider the composite number n = 81317

Consider the different ways we have seen that might help us show this (use Maple as necessary):

a)	How many trial divisions by small primes do you have to carry out before finding a factor?

In [ ]:
n = 81317

print([f for f in number_theory.ifactors(n)])
for i, p in enumerate(number_theory.primes_below(math.trunc(math.sqrt(n) + 1))):
    if n % p == 0:
        print(f"a) {i} trial divisions to find that {n} is composite: {p} x {n // p} = {p * n // p}")
        break

b)	Choose values of $a$ at random using Maple – how many do you need before the gcd test shows n is composite?

In [ ]:
n = 81317

i = 0
while True:
    a = secrets.randbelow(n)
    i += 1
    gcd = number_theory.gcd(n, a)
    if gcd > 1:
        break
        
print(f"b) {i} rand numbers before finding that {n} is composite: {gcd} x {int(n / gcd)} = {gcd * int(n / gcd)}")

c)	How many values of $a$ do you need before you find a Fermat witness?

Try to calculate the proportion of Fermat witnesses and Fermat Liars for this number n.

In [ ]:
def is_fermat_prime(a:int, n: int):
    return pow(a, n - 1, n) == 1

def fermat_witnesses(n: int, stop_on_first_witness=True):
    fermat_liars = 0
    fermat_witnesses = 0
    first_witness_count = 0
    for a in range(2, n - 1):
        if math.gcd(a, n) == 1:
            if not is_fermat_prime(a, n):
                first_witness_count = first_witness_count or a - 1
                fermat_witnesses += 1
                if stop_on_first_witness:
                    break
            else:
                fermat_liars += 1
    
    if first_witness_count:
        print(f"{a - 1} rounds to find:")
    else:
        print(f"No witness found in {n - 2} rounds:")
    print(f"Fermat witnesses: {fermat_witnesses}\tFermat Liars: {fermat_liars}")
    return first_witness_count

In [ ]:
n = 81317
print(f"c) {fermat_witnesses(n, stop_on_first_witness=True)} tests before finding a fermat witness for {n}")

__2. Show that $1729 = 7 * 13 * 19$ is a Carmichael Number using two methods:__

i)  by showing directly that it has no Fermat Witnesses

In [ ]:
n = 7 * 13 * 19
print(number_theory.ifactors(13))
print(f"{n} = {number_theory.ifactors(n)} and is a fermat prime {all([is_fermat_prime(n, a) for a in range(2, n) if math.gcd(a, n) == 1])}")

fermat_witnesses(n)

ii) by showing that it satisfies Korselt’s Criteria:  

* $n | a^n-a$ for all integers $a$ iff:
    1. $n$ is squarefree and 
    1. $(p-1)|(n-1)$ for all prime divisors $p$ of $n$. 
    
Carmichael numbers satisfy this criterion. 

In [ ]:
def is_fermat_prime(n: int, a: int) -> bool:
    return pow(a, n - 1, n) == 1

print(f"\nn = {n} satisfies FLT for all a where (a, n) = 1 with 1 < a < n:")
print(f"n divides a^n - a for all integers 1 < a < n: \t{all([(a ** n - a) % n == 0 for a in range(2, n)])}")
print(f"n divides a^n - a for all integers 1 < a < n: \t{all([(a ** n) % n == a for a in range(2, n)])}")
print(f"n divides a^n - a for all integers 1 < a < n: \t{all([is_fermat_prime(n, a) for a in range(2, n) if math.gcd(a, n) == 1])}")

print(f"\nn = {n} is square free (by construction): \t{all([n % d ** 2 != 0 for d in range(2, int(np.sqrt(n) + 1))])}")
print(f"(p - 1) | (n - 1) for all prime divisors: \t{all([(n - 1) % (p - 1) == 0 for p in number_theory.primes_below(n) if n % p == 0])}")

__What else is remarkable about the number 1729 (google ‘Ramanujan and Hardy 1729’ or similar).__

$Taxicab(n)$ is defined as the smallest integer that can be expressed as a sum of two positive integer cubes in $n$ distinct ways.   Of $1792$, "it is a very interesting number; it is the smallest number expressible as the sum of two [positive] cubes in two different ways."

__3.	Write a Maple procedure implementing Korselt’s Criteria; you can use the maple command issqrfree (part of the Number Theory package) if you have to but it would be better to do both steps yourself.__

In [ ]:
def is_korselt_criteria(n):
    """ n divides a^n - a for all integers a if and only if:
        i)    n is squarefree (i.e. n is not divisible by d^2 for any d) 
        ii)   p | n => (p-1) | (n-1) """
    return all([n % (d ** 2) != 0 for d in range(2, int(np.sqrt(n)) + 1)]) \
        and all([(n - 1) % (p - 1) == 0 for p in number_theory.primes_below(n) if n % p == 0])

__4.	There is one other Carmichael Number between 561 and 1729. What is it?__

A Carmichael number is an odd composite number n which satisfies Fermat's little theorem: $a^{n-1} - 1 \equiv 0 \pmod{n}$ for every choice of a satisfying $gcd(a, n) = 1$.

In [ ]:
# A composite number n is a Carmichael Number if it satisfys FLT
for i in range(561, 1730):
    if not number_theory.is_prime(i) and is_korselt_criteria(i):
        print(f"{i} is a Carmichael number")

# Directly checking FLT takes a long time
for i in range(561, 1730):
    if not number_theory.is_prime(i) and all([pow(a, i, i) == a for a in range(2, i)]):
        print(f"Composite {i} does satisfy FLT for all a where (a, {i}) = 1 with 1 < a < {i}.")

__5.     	Suppose we are looking at the number m = 221 (which is composite as 221 = 13 * 17). How many choices of a in the range [2, 219] will cause the Miller Rabin Test to wrongly return the answer that m is probably prime.__

In [ ]:
def is_miller_rabin_prime(n, k=5):
    """  Miller-Rabin primality test to check if a number is probably prime.  """
    if n <= 1:
        return False, 0
    if n <= 3:
        return True, 0

    # Write n as (2^r) * d + 1
    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2

    # Witness loop
    for i in range(k):
        a = secrets.randbelow(n - 3) + 2
        x = pow(a, d, n)
        for _ in range(s - 1):
            y = x ** 2 % n
            if y == 1 and x != 1 and x != n - 1:
                return False, i + 1
            x = y
            
        if y != 1:
            # Definitely composite
            return False, i + 1

    return True, k

In [ ]:
m = 221
is_probably_prime, iter_count = is_miller_rabin_prime(m, 10)
print(f"{m} is {'' if is_probably_prime else 'not '}prime after {iter_count} iterations.")

__6.	561 (our first Carmichael Number) cannot be shown to be composite with the Fermat Test. What happens if you try to examine its primality using the Miller Rabin test instead?__

In [ ]:
m = 561
is_probably_prime, iter_count = is_miller_rabin_prime(m, 10)
print(f"{m} is {'' if is_probably_prime else 'not '}prime after {iter_count} iterations.")

__7.	Now look at the number $m = 247$; if we choose a number a in the range $[2, 245]$ which is more likely to show that m is composite?__ 

		i) a Fermat Test calculating $a^{m-1} mod m$
        
		ii) a gcd test on a and m
        
		iii) a Miller Rabin test 
        
		iv) Trial division/sieving methods
        
Thinking about the probability that the test produces a clear answer, the amount of work that it involves and any other relevant factors rank the four tests in order of preference for writing a short explanation of your reasons.   

Does the size of the target number affect your answer? Does it change for:

a)	Numbers less than $10,000,000$

b)	Numbers bigger than $1,000,000,000,000$


In [ ]:
def is_fermat_composite(n: int, a: int) -> bool:
    return pow(a, n - 1, n) != 1

def is_gcd_composite(n: int, a: int) -> bool:
    return math.gcd(a, n) != 1

def is_miller_rabin_composite(n: int, a: int) -> bool:
    if n == 2 or n == 3:
        return False
    if n <= 1 or n % 2 == 0:
        return True

    # Write (n - 1) as 2^s * d
    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2

    x = pow(a, d, n) # a^d % n
    if x == 1 or x == n - 1:
        return False

    for _ in range(s - 1):
        x = pow(x, 2, n)
        if x == n - 1:
            break
    else:
        return True

def is_trial_division_composite(n: int, a: int) -> bool:
    if number_theory.is_prime(a) and n % a == 0:
        return True
    return False

def run_trial(m, trials, is_composite_fn, verbose=True):
    composite_count = 0
    for _ in range(trials):
        a = secrets.randbelow(m - 3) + 2
        composite_count += 1 if is_composite_fn(m, a) else 0
    if verbose:
        print(composite_count / trials)
    return composite_count / trials

m = 247
trials = 100
print(f"{m} = {number_theory.ifactors(m)} is prime {number_theory.is_prime(m)}")
run_trial(m, trials, is_fermat_composite)
run_trial(m, trials, is_gcd_composite)
run_trial(m, trials, is_miller_rabin_composite)
run_trial(m, trials, is_trial_division_composite)
# print(sum([1 for a in range(2, m - 2) if is_fermat_composite(m, a)]) / (m - 3))
# print(sum([1 for a in range(2, m - 2) if is_gcd_composite(m, a)]) / (m - 3))
# print(sum([1 for a in range(2, m - 2) if is_miller_rabin_composite(m, a)]) / (m - 3))
# print(sum([1 for a in range(2, m - 2) if is_trial_division_composite(m, a)]) / (m - 3))


m = number_theory.sieve_of_eratosthenes(10000)[-1] + 4
trials = 100
print(f"{m} = {number_theory.ifactors(m)} is prime {number_theory.is_prime(m)}")
run_trial(m, trials, is_fermat_composite)
run_trial(m, trials, is_gcd_composite)
run_trial(m, trials, is_miller_rabin_composite)
run_trial(m, trials, is_trial_division_composite)
print(sum([1 for a in range(2, m - 2) if is_fermat_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_gcd_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_miller_rabin_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_trial_division_composite(m, a)]) / (m - 3))

m = 11 * 13 * 17 * 307
trials = 100
print(f"{m} = {number_theory.ifactors(m)} is prime {number_theory.is_prime(m)}")
run_trial(m, trials, is_fermat_composite)
run_trial(m, trials, is_gcd_composite)
run_trial(m, trials, is_miller_rabin_composite)
run_trial(m, trials, is_trial_division_composite)
print(sum([1 for a in range(2, m - 2) if is_fermat_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_gcd_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_miller_rabin_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_trial_division_composite(m, a)]) / (m - 3))

m = 11 * 13 * 17 * 97 * 307
trials = 100
print(f"{m} = {number_theory.ifactors(m)} is prime {number_theory.is_prime(m)}")
run_trial(m, trials, is_fermat_composite)
run_trial(m, trials, is_gcd_composite)
run_trial(m, trials, is_miller_rabin_composite)
run_trial(m, trials, is_trial_division_composite)
print(sum([1 for a in range(2, m - 2) if is_fermat_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_gcd_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_miller_rabin_composite(m, a)]) / (m - 3))
print(sum([1 for a in range(2, m - 2) if is_trial_division_composite(m, a)]) / (m - 3))

In [ ]:
%%timeit
run_trial(m, trials, is_fermat_composite, verbose=False)

In [ ]:
%%timeit
run_trial(m, trials, is_gcd_composite, verbose=False)

In [ ]:
%%timeit
run_trial(m, trials, is_miller_rabin_composite, verbose=False)

In [ ]:
%%timeit
run_trial(m, trials, is_trial_division_composite, verbose=False)

__8.	Read the last slide for the week 2 lecture; identify any terms that you aren’t familiar with and try to find definitions of these concepts before we look at the algorithm in detail.__

In [ ]:
%%timeit -n100
primes(N);

In [ ]:
import random

def miller_rabin_composite(n, a):
    if n == 2 or n == 3:
        return False
    if n <= 1 or n % 2 == 0:
        return True

    # Write (n - 1) as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    x = pow(a, d, n) # a^d % n
    if x == 1 or x == n - 1:
        return False

    for _ in range(r - 1):
        x = pow(x, 2, n)
        if x == n - 1:
            break
    else:
        return True

    return False
def miller_rabin(n, k):
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False

    # Write (n - 1) as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n) # a^d % n
        if x == 1 or x == n - 1:
            continue

        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True

# Example usage
n = 13 # Replace with the number you want to test
k = 5  # Number of iterations, higher is more accurate
print(miller_rabin(n, k))
print([ miller_rabin_composite(13, a) for a in range(2,12)])